In [1]:
import pandas as pd
import csv
import re
from html.parser import HTMLParser
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier #DeprecationWarning on numpy.core.umath_tests
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid

D:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# descripton cleaning helper functions

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def remove_email(desc):
    match = re.search(r'[\w\.-]+@[\w\.-]+', desc)
    
    if match is not None:
        email = match.group(0)
        # print(f'EMAIL={email}')
        return desc.replace(email,'')
    else:
        # print('NO EMAIL IN DESCRIPTION')
        return desc

def clean_description(html):
    #desc = unidecode.unidecode(html)
    desc = unicodedata.normalize('NFKD', html)
    
    # remove email
    desc = remove_email(desc)
    
    # replace linebreaks and paragraph tags with space
    desc = re.sub('<\\s*br\\s*>', ' ', desc)
    desc = re.sub('<\\s*br\\s*/\\s*>', ' ', desc)
    desc = re.sub('<\\s*p\\s*>', ' ', desc)
    desc = re.sub('<\\s*p\\s*/\\s*>', ' ', desc)
    
    # parse and remove other HTML tags / symbol entities
    desc = strip_tags(desc)
    
    # replace all non-alphanumeric characters
    # may or may not bite us in the ass; see how it goes
    desc = re.sub('[^a-zA-Z0-9]', ' ', desc)
    
    # lowercase result string
    desc = desc.lower()
    
    # return cleaned string with extra spaces removed
    return ' '.join(desc.split())

In [3]:
# increases text displayed in pandas table for Jupyter Notebook
pd.options.display.max_colwidth = 10000

# read csv into pandas DataFrame (sample.csv = 100000)
df = pd.read_csv('sample.csv', keep_default_na=False)

# remove listings where description is blank (100000 -> 99680)
df = df[df['description'] != '']

# clean descriptions (takes some time) and remove blanks after cleaning (99680 -> 99641)
df['cleaned'] = df['description'].apply(clean_description)
df = df[df['cleaned'] != '']

In [4]:
# 70:30 train-test split; 99641 * 0.7 ~ 69750
df_train = df[0:69750]
df_test = df[69750:]

# training description and label
df_train_desc = df_train['cleaned']
df_train_label = df_train['onet']

# testing description and label
df_test_desc = df_test['cleaned']
df_test_label = df_test['onet']

In [5]:
# initialize count vectorizer
count_vectorizer = CountVectorizer(strip_accents='unicode', stop_words='english', ngram_range=(1,1))
df_train_cv = count_vectorizer.fit_transform(df_train_desc)

# initialize tf transformer
tf_transformer = TfidfTransformer(use_idf=False)
df_train_tf = tf_transformer.fit_transform(df_train_cv)

# initialize tf-idf transformer
tfidf_transformer = TfidfTransformer(use_idf=True)
df_train_tfidf = tfidf_transformer.fit_transform(df_train_cv)

In [6]:
#df_test_tf = get_test_tf_transformer(df_train_cv, df_train_tfidf, df_test['cleaned'])
df_test_cv = count_vectorizer.transform(df_test_desc)
df_test_tf = tf_transformer.transform(df_test_cv)
df_test_tfidf = tfidf_transformer.transform(df_test_cv)

# Multinomial Naive Bayes

In [7]:
"""
# test multinomial naive bayes classification (term frequency)
mnb_clf_tf = MultinomialNB().fit(df_train_tf, df_train_label)
#predicted = clf.predict(df_test_tfidf)
score = mnb_clf_tf.score(df_test_tf, df_test_label)
score
"""

'\n# test multinomial naive bayes classification (term frequency)\nmnb_clf_tf = MultinomialNB().fit(df_train_tf, df_train_label)\n#predicted = clf.predict(df_test_tfidf)\nscore = mnb_clf_tf.score(df_test_tf, df_test_label)\nscore\n'

In [8]:
"""
# test naive bayes classification (tf-idf)
mnb_clf_tfidf = MultinomialNB().fit(df_train_tfidf, df_train_label)
score = mnb_clf_tfidf.score(df_test_tfidf, df_test_label)
score
"""

'\n# test naive bayes classification (tf-idf)\nmnb_clf_tfidf = MultinomialNB().fit(df_train_tfidf, df_train_label)\nscore = mnb_clf_tfidf.score(df_test_tfidf, df_test_label)\nscore\n'

# Ridge Regression Classification

In [11]:
"""
# test ridge regression classifier (tf)
rr_clf_tf = RidgeClassifier(max_iter=1).fit(df_train_tf, df_train_label)
score = rr_clf_tf.score(df_test_tf, df_test_label)
score
"""

D:\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [14]:
"""
# test ridge regression classifier (tf-idf)
rr_clf_tfidf = RidgeClassifier(max_iter=1).fit(df_train_tfidf, df_train_label)
score = rr_clf_tfidf.score(df_test_tfidf, df_test_label)
score
"""

D:\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.6406276136629755

# Random Forest Classification

In [15]:
"""
# test random forest classification (tf)
rf_clf_tf = RandomForestClassifier().fit(df_train_tf, df_train_label)
score = rf_clf_tf.score(df_test_tf, df_test_label)
score
"""

0.5836539426583253

In [16]:
"""
# test random forest classification (tf-idf)
rf_clf_tfidf = RandomForestClassifier().fit(df_train_tfidf, df_train_label)
score = rf_clf_tfidf.score(df_test_tfidf, df_test_label)
score
"""

0.582081563012278

# Linear SVC Classification

In [18]:
"""
# test linear SVC classification (tf)
lsvc_clf_tf = LinearSVC().fit(df_train_tf, df_train_label)
score = lsvc_clf_tf.score(df_test_tf, df_test_label)
score
"""

0.6930848750460005

In [19]:
"""
lsvc_clf_tfidf = LinearSVC().fit(df_train_tfidf, df_train_label)
score = lsvc_clf_tfidf.score(df_test_tfidf, df_test_label)
score
"""

0.6958616305911478

# Linear w/ Stochastic Gradient Descent

In [21]:
"""
sgd_clf_tf = SGDClassifier().fit(df_train_tf, df_train_label)
score = sgd_clf_tf.score(df_test_tf, df_test_label)
score
"""

D:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.6417316249038172

In [22]:
"""
sgd_clf_tfidf = SGDClassifier().fit(df_train_tfidf, df_train_label)
score = sgd_clf_tfidf.score(df_test_tfidf, df_test_label)
score
"""

D:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.6568866883008263

# Perceptron Classification

In [24]:
p_clf_tf = SGDClassifier().fit(df_train_tf, df_train_label)
score = p_clf_tf.score(df_test_tf, df_test_label)
score

D:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.64136362115687

In [25]:
p_clf_tfidf = SGDClassifier().fit(df_train_tfidf, df_train_label)
score = p_clf_tfidf.score(df_test_tfidf, df_test_label)
score

D:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.6549463049078318

# Nearest Centroid

In [10]:
# ‘cityblock’, ‘cosine’, ‘euclidean’, ‘l1’, ‘l2’, ‘manhattan’
nc_clf_tf = NearestCentroid(metric='cosine').fit(df_train_tf, df_train_label)
score = nc_clf_tf.score(df_test_tf, df_test_label)
score

D:\Anaconda3\lib\site-packages\sklearn\neighbors\nearest_centroid.py:140: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


0.5040647686594627

In [13]:
nc_clf_tfidf = NearestCentroid(metric='cosine').fit(df_train_tfidf, df_train_label)
score = nc_clf_tfidf.score(df_test_tfidf, df_test_label)
score

D:\Anaconda3\lib\site-packages\sklearn\neighbors\nearest_centroid.py:140: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


0.5342076210230504